<a href="https://colab.research.google.com/github/drago467/Deep-Facial-Recognition-App/blob/main/Facial%20Verification%20with%20a%20Siamese%20Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Import standard dependencies
import cv2
import os
import random
import numpy as np
from matplotlib import pyplot as plt

In [2]:
# Import tensorflow depencies - Functional API
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer, Conv2D, Dense, MaxPooling2D, Input, Flatten
import tensorflow as tf

In [3]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

In [4]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [5]:
# Setup paths
POS_PATH = os.path.join('data', 'positive')
NEG_PATH = os.path.join('data', 'negative')
ANC_PATH = os.path.join('data', 'anchor')

In [6]:
# Make the directories
os.makedirs(POS_PATH)
os.makedirs(NEG_PATH)
os.makedirs(ANC_PATH)

In [7]:
# Uncompress Tar GZ Labelled Faces in the Wild Dataset

!wget http://vis-www.cs.umass.edu/lfw/lfw.tgz

!tar -xf lfw.tgz

--2024-09-14 14:07:42--  http://vis-www.cs.umass.edu/lfw/lfw.tgz
Resolving vis-www.cs.umass.edu (vis-www.cs.umass.edu)... 128.119.244.95
Connecting to vis-www.cs.umass.edu (vis-www.cs.umass.edu)|128.119.244.95|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 180566744 (172M) [application/x-gzip]
Saving to: ‘lfw.tgz’

lfw.tgz             100%[===================>] 172.20M  1.48MB/s    in 2m 23s  

2024-09-14 14:10:05 (1.21 MB/s) - ‘lfw.tgz’ saved [180566744/180566744]



In [11]:
# Move LFW Images to the following repository data/negative
for directory in os.listdir('lfw'):
    # Tạo đường dẫn đầy đủ đến thư mục con
    directory_path = os.path.join('lfw', directory)

    # Kiểm tra nếu đường dẫn là một thư mục
    if os.path.isdir(directory_path):
        for file in os.listdir(directory_path):
            EX_PATH = os.path.join(directory_path, file)
            NEW_PATH = os.path.join(NEG_PATH, file)
            os.replace(EX_PATH, NEW_PATH)

In [13]:
 # Import uuid library to generate unique image names
 import uuid

 # Establish a connection to the webcam
 cap = cv2.VideoCapture(4)
 while cap.isOpened():
  ret, frame = cap.read()

  # Cut down  frame to 250x250px
  frame = frame[120:120+250,200:200+250, :]

  # Collect anchors
  if cv2.waitKey(1) & 0XFF == ord('a'):
    imgname = os.path.join(ANC_PATH, '{}.jpg'.format(uuid.uuid1()))
    cv2.imwrite(imgname, frame)

  # Collect possitives
  if cv2.waitKey(1) & 0XFF == ord('p'):
    imgname = os.path.join(POS_PATH, '{}.jpg'.format(uuid.uuid1()))
    cv2.imwrite(imgname, frame)

  # Show image back to screen
  cv2.imshow('Image Collection', frame)

  # Breaking gracefully
  if cv2.waikey(1) & OXFF == ord('q'):
    break

# Release the webcam
cap.release()
# Close the image show frame
cv2.destroyAllWindows()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [14]:
import os

# Tạo thư mục đích nếu chưa tồn tại
destination_dir = '/content/data/anchor'
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Sao chép tất cả các tệp ảnh (.jpg, .jpeg, .png) từ Google Drive
!cp /content/drive/MyDrive/anchor/*.{jpg,jpeg,png} /content/data/anchor/


cp: cannot stat '/content/drive/MyDrive/anchor/*.jpeg': No such file or directory
cp: cannot stat '/content/drive/MyDrive/anchor/*.png': No such file or directory


In [15]:
import os

# Tạo thư mục đích nếu chưa tồn tại
destination_dir = '/content/data/positive'
if not os.path.exists(destination_dir):
    os.makedirs(destination_dir)

# Sao chép tất cả các tệp ảnh (.jpg, .jpeg, .png) từ Google Drive
!cp /content/drive/MyDrive/positive/*.{jpg,jpeg,png} /content/data/positive/


cp: cannot stat '/content/drive/MyDrive/positive/*.jpeg': No such file or directory
cp: cannot stat '/content/drive/MyDrive/positive/*.png': No such file or directory


In [17]:
anchor = tf.data.Dataset.list_files(ANC_PATH+'/*.jpg').take(300)
positive = tf.data.Dataset.list_files(POS_PATH+'/*.jpg').take(300)
negative = tf.data.Dataset.list_files(NEG_PATH+'/*.jpg').take(300)

In [ ]:
def preprocess(file_path):


In [18]:
dir_test = anchor.as_numpy_iterator()
dir_test.next()

b'data/anchor/1943c7eb-72af-11ef-bc5b-701ab85ee0c9.jpg'

#**3.2 Preprocessing - Scale and Resize**

In [ ]:
def preprocess(file_path):
  byte_img = tf.io.read_file(file_path)
  img = tf.io.decode_jpeg(byte_img)
  img = tf.image.resize(img, (105,105))
  img = img / 255.0
  return img

# **3.3 Create Labelled Dataset**

In [19]:
positives = tf.data.Dataset.zip((anchor, positive, tf.data.Dataset.from_tensor_slices(tf.ones(len(anchor)))))
negatives = tf.data.Dataset.zip((anchor, negative, tf.data.Dataset.from_tensor_slices(tf.zeros(len(anchor)))))
data = positives.concatenate(negatives)


In [20]:
samples = data.as_numpy_iterator()

In [21]:
samples.next()

(b'data/anchor/60b5a8c9-72af-11ef-ae7b-701ab85ee0c9.jpg',
 b'data/positive/21695cb3-72b0-11ef-8d2b-701ab85ee0c9.jpg',
 1.0)